In [1]:
from datetime import date, datetime
from dateutil import parser
from io import StringIO
import pandas as pd
import requests, perspective, json

In [2]:
# standardize on abbreviations for states, and full names in 'stateName'
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP"
}
us_state_full = {key: value for (value, key) in us_state_abbrev.items()}

In [3]:
STATE_URL = "https://covidtracking.com/api/states/daily"
CONFIRMED_URL = "https://static.usafacts.org/public/data/covid-19/covid_confirmed_usafacts.csv"
DEATHS_URL = "https://static.usafacts.org/public/data/covid-19/covid_deaths_usafacts.csv"

### Clean Up State Level Data

In [4]:
def clean_state_data(data):
    for row in data:
        # dates are stored as integer with no separators, so add separators and parse
        d = list(str(row["date"]))
        if "/" not in d:
            d.insert(4, "/")
            d.insert(7, "/")
        row["date"] = "".join(d)
        row["date"] = parser.parse(row["date"])
        row["dateChecked"] = parser.parse(row["dateChecked"])
    return data

In [5]:
state_data = clean_state_data(requests.get(STATE_URL).json())

In [6]:
state_aggregates = {
    "positive": "high",
    "negative": "high",
    "pending": "high",
    "death": "high",
    "total": "high"
}

In [7]:
state_data = pd.DataFrame(state_data).rename(columns={
    "state": "State"
})

In [8]:
state_data["stateName"] = [us_state_full.get(x, None) for x in state_data["State"]]

In [9]:
state_data

,date,State,positive,negative,pending,hospitalized,death,total,dateChecked,stateName
0,2020-03-23,AK,22.0,946.0,NaN,0.0,NaN,968,2020-03-23 20:00:00+00:00,Alaska
1,2020-03-23,AL,167.0,1665.0,NaN,NaN,0.0,1832,2020-03-23 20:00:00+00:00,Alabama
2,2020-03-23,AR,174.0,906.0,0.0,13.0,0.0,1080,2020-03-23 20:00:00+00:00,Arkansas
3,2020-03-23,AS,NaN,NaN,NaN,NaN,0.0,0,2020-03-23 20:00:00+00:00,American Samoa
4,2020-03-23,AZ,265.0,309.0,6.0,NaN,2.0,580,2020-03-23 20:00:00+00:00,Arizona
...,...,...,...,...,...,...,...,...,...,...
976,2020-03-04,OR,3.0,29.0,18.0,NaN,NaN,50,2020-03-04 21:00:00+00:00,Oregon
977,2020-03-04,SC,0.0,5.0,0.0,NaN,NaN,5,2020-03-04 21:00:00+00:00,South Carolina
978,2020-03-04,TX,1.0,NaN,NaN,NaN,NaN,1,2020-03-04 21:00:00+00:00,Texas
979,2020-03-04,WA,39.0,NaN,NaN,NaN,NaN,39,2020-03-04 21:00:00+00:00,Washington


### Clean up and reorganize county level data

In [10]:
confirmed = StringIO(requests.get(CONFIRMED_URL).text)
deaths = StringIO(requests.get(DEATHS_URL).text)

Dates are stored as column names in these CSVs, so we need to transform them into row values and rename the columns accordingly.

In [11]:
confirmed_df = pd.read_csv(confirmed).melt(id_vars=["countyFIPS", "County Name", "State", "stateFIPS"])
confirmed_df = confirmed_df.rename(columns={"variable": "Date", "value": "Confirmed", "County Name": "County"})
confirmed_df["stateName"] = [us_state_full.get(x, None) for x in confirmed_df["State"]]

In [12]:
confirmed_df

,countyFIPS,County,State,stateFIPS,Date,Confirmed,stateName
0,0,Statewide Unallocated,AL,1,1/22/2020,0,Alabama
1,1003,Baldwin County,AL,1,1/22/2020,0,Alabama
2,1015,Calhoun County,AL,1,1/22/2020,0,Alabama
3,1017,Chambers County,AL,1,1/22/2020,0,Alabama
4,1043,Cullman County,AL,1,1/22/2020,0,Alabama
...,...,...,...,...,...,...,...
72036,56021,Laramie County,WY,56,3/22/2020,6,Wyoming
72037,56025,Natrona County,WY,56,3/22/2020,1,Wyoming
72038,56029,Park County,WY,56,3/22/2020,1,Wyoming
72039,56033,Sheridan County,WY,56,3/22/2020,4,Wyoming


In [13]:
deaths_df = pd.read_csv(deaths)
deaths_df = deaths_df.melt(id_vars=["ï»¿countyFIPS", "County Name", "State", "stateFIPS"])
deaths_df = deaths_df.rename(columns={"ï»¿countyFIPS": "countyFIPS", "variable": "Date", "value": "Deaths", "County Name": "County"})
deaths_df["stateName"] = [us_state_full.get(x, None) for x in deaths_df["State"]]

In [14]:
deaths_df

,countyFIPS,County,State,stateFIPS,Date,Deaths,stateName
0,0,Statewide Unallocated,AL,1,1/22/2020,0,Alabama
1,1003,Baldwin County,AL,1,1/22/2020,0,Alabama
2,1015,Calhoun County,AL,1,1/22/2020,0,Alabama
3,1017,Chambers County,AL,1,1/22/2020,0,Alabama
4,1043,Cullman County,AL,1,1/22/2020,0,Alabama
...,...,...,...,...,...,...,...
72036,56021,Laramie County,WY,56,3/22/2020,0,Wyoming
72037,56025,Natrona County,WY,56,3/22/2020,0,Wyoming
72038,56029,Park County,WY,56,3/22/2020,0,Wyoming
72039,56033,Sheridan County,WY,56,3/22/2020,0,Wyoming


### Get auxillary state & county-level data

In [15]:
# Closest estimates we have are from 2018
STATE_POPULATION_URL = "http://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-popchg2010_2019.csv?#"
COUNTY_POPULATION_URL = "https://www.ers.usda.gov/webdocs/DataFiles/48747/PopulationEstimates.csv?v=3011.3"
COUNTY_UNEMPLOYMENT_URL = "https://www.ers.usda.gov/webdocs/DataFiles/48747/Unemployment.csv?v=2564.4"

In [16]:
state_population = StringIO(requests.get(STATE_POPULATION_URL).text)
state_population_df = pd.read_csv(
    state_population, usecols=["STATE", "NAME", "POPESTIMATE2019"]) \
        .rename(columns={"POPESTIMATE2019": "Population (2019 Estimate)", "NAME": "stateName", "STATE": "stateFIPS"}) \
        .set_index("stateFIPS")

In [17]:
state_population_df["State"] = [us_state_abbrev.get(x, None) for x in state_population_df["stateName"]]

In [18]:
state_population_df = state_population_df[state_population_df["State"].notnull()]

In [19]:
# cleaned and normalized
state_population_df

,stateName,Population (2019 Estimate),State
stateFIPS,,,
1,Alabama,4903185,AL
2,Alaska,731545,AK
4,Arizona,7278717,AZ
5,Arkansas,3017804,AR
6,California,39512223,CA
8,Colorado,5758736,CO
9,Connecticut,3565287,CT
10,Delaware,973764,DE
11,District of Columbia,705749,DC


In [20]:
county_population = StringIO(requests.get(COUNTY_POPULATION_URL).text)
county_population_df = pd.read_csv(
    county_population, usecols=["FIPS", "State", "Area_Name", "POP_ESTIMATE_2018"]) \
    .rename(columns={"POP_ESTIMATE_2018": "Population (2018 Estimate)", "Area_Name": "County", "FIPS": "countyFIPS"}) \
    .set_index("countyFIPS")

In [21]:
county_population_df["stateName"] = [us_state_full.get(x, None) for x in county_population_df["State"]]

In [22]:
county_population_df = county_population_df[county_population_df["stateName"].notnull()]

In [23]:
county_population_df["Population (2018 Estimate)"] = pd.to_numeric(county_population_df["Population (2018 Estimate)"].str.replace(",","").astype(float))

In [24]:
county_population_df

,State,County,Population (2018 Estimate),stateName
countyFIPS,,,,
1000.0,AL,Alabama,4887871.0,Alabama
1001.0,AL,Autauga County,55601.0,Alabama
1003.0,AL,Baldwin County,218022.0,Alabama
1005.0,AL,Barbour County,24881.0,Alabama
1007.0,AL,Bibb County,22400.0,Alabama
...,...,...,...,...
72145.0,PR,"Vega Baja Municipio, Puerto Rico",50185.0,Puerto Rico
72147.0,PR,"Vieques Municipio, Puerto Rico",8364.0,Puerto Rico
72149.0,PR,"Villalba Municipio, Puerto Rico",21476.0,Puerto Rico


In [25]:
# Fold NYC's counties into one value under FIPS 36061, which belongs to Manhattan
nyc_counties = county_population_df[county_population_df["County"].isin(["Kings County", "Queens County", "New York County", "Bronx County", "Richmond County"])]

In [26]:
nyc_counties = nyc_counties[nyc_counties["State"] == "NY"]

In [27]:
nyc_counties

,State,County,Population (2018 Estimate),stateName
countyFIPS,,,,
36005.0,NY,Bronx County,1432132.0,New York
36047.0,NY,Kings County,2582830.0,New York
36061.0,NY,New York County,1628701.0,New York
36081.0,NY,Queens County,2278906.0,New York
36085.0,NY,Richmond County,476179.0,New York


In [28]:
nyc_pop = nyc_counties["Population (2018 Estimate)"].sum()

In [29]:
nyc_population = pd.DataFrame([{
    "countyFIPS": 36061,
    "State": "NY",
    "County": "New York City",
    "Population (2018 Estimate)": nyc_pop,
    "stateName": "New York"
}]).set_index("countyFIPS")

In [30]:
county_population_df = county_population_df.drop([36005, 36047, 36081, 36085])

In [31]:
county_population_df = county_population_df.append(nyc_population, sort=True)

In [32]:
county_population_df = county_population_df.reset_index()

In [33]:
county_unemployment = StringIO(requests.get(COUNTY_UNEMPLOYMENT_URL).text)
county_unemployment_df = pd.read_csv(
    county_unemployment, usecols=["FIPS", "State", "Area_name", "Unemployment_rate_2018", "Median_Household_Income_2018", "Civilian_labor_force_2018", "Employed_2018", "Unemployed_2018"]) \
    .rename(columns={
            "Unemployment_rate_2018": "Unemployment Rate (2018)",
            "Civilian_labor_force_2018": "Civilian Labor Force (2018)",
            "Employed_2018": "Employed (2018)",
            "Unemployed_2018": "Unemployed (2018)",
            "Median_Household_Income_2018": "Median Household Income (2018)",
            "Area_name": "County",
            "FIPS": "countyFIPS"
        }) \
    .set_index("countyFIPS")

In [34]:
for col in ["Civilian Labor Force (2018)", "Employed (2018)", "Unemployed (2018)"]:
    county_unemployment_df[col] =  pd.to_numeric(county_unemployment_df[col].str.replace(",","").astype(float))

In [35]:
county_unemployment_df["Median Household Income (2018)"] =  pd.to_numeric([None if str(x) == "nan" else str(x).replace(",","").replace("$","") for x in county_unemployment_df["Median Household Income (2018)"]])

In [36]:
county_unemployment_df

,State,County,Civilian Labor Force (2018),Employed (2018),Unemployed (2018),Unemployment Rate (2018),Median Household Income (2018)
countyFIPS,,,,,,,
0,US,United States,161640488.0,155344043.0,6296445.0,3.9,61937.0
1000,AL,Alabama,2198837.0,2112347.0,86490.0,3.9,49881.0
1001,AL,"Autauga County, AL",25957.0,25015.0,942.0,3.6,59338.0
1003,AL,"Baldwin County, AL",93849.0,90456.0,3393.0,3.6,57588.0
1005,AL,"Barbour County, AL",8373.0,7940.0,433.0,5.2,34382.0
...,...,...,...,...,...,...,...
72145,PR,"Vega Baja Municipio, PR",13199.0,11826.0,1373.0,10.4,NaN
72147,PR,"Vieques Municipio, PR",2734.0,2283.0,451.0,16.5,NaN
72149,PR,"Villalba Municipio, PR",7329.0,6164.0,1165.0,15.9,NaN


In [37]:
# Fold NYC's counties into one value under FIPS 36061, which belongs to Manhattan
nyc_counties_meta = county_unemployment_df[county_unemployment_df["County"].isin(["Kings County, NY", "Queens County, NY", "New York County, NY", "Bronx County, NY", "Richmond County, NY"])]

In [38]:
nyc_counties_meta = nyc_counties_meta[nyc_counties_meta["State"] == "NY"]

In [39]:
nyc_counties_meta

,State,County,Civilian Labor Force (2018),Employed (2018),Unemployed (2018),Unemployment Rate (2018),Median Household Income (2018)
countyFIPS,,,,,,,
36005,NY,"Bronx County, NY",605864.0,571545.0,34319.0,5.7,38566.0
36047,NY,"Kings County, NY",1211721.0,1160501.0,51220.0,4.2,60862.0
36061,NY,"New York County, NY",919101.0,885351.0,33750.0,3.7,84610.0
36081,NY,"Queens County, NY",1162225.0,1120459.0,41766.0,3.6,68432.0
36085,NY,"Richmond County, NY",220621.0,211509.0,9112.0,4.1,79719.0


In [40]:
nyc_meta_all = pd.DataFrame([{
    "countyFIPS": 36061,
    "State": "NY",
    "County": "New York City",
    "Civilian Labor Force (2018)": nyc_counties_meta["Civilian Labor Force (2018)"].sum(),
    "Employed (2018)": nyc_counties_meta["Employed (2018)"].sum(),
    "Unemployed (2018)": nyc_counties_meta["Unemployed (2018)"].sum(),
    "Unemployment Rate (2018)": nyc_counties_meta["Unemployment Rate (2018)"].sum(),
    "Median Household Income (2018)": nyc_counties_meta["Median Household Income (2018)"].sum(),
    "stateName": "New York"
}]).set_index("countyFIPS")

In [41]:
county_unemployment_df = county_unemployment_df.drop([36005, 36047, 36081, 36085])
county_unemployment_df = county_unemployment_df.append(nyc_meta_all, sort=True).reset_index()

### Join together all datasets

In [42]:
state_covid_with_population = state_data.merge(state_population_df[["Population (2019 Estimate)", "State"]], on="State")

In [43]:
state_covid_with_population

,date,State,positive,negative,pending,hospitalized,death,total,dateChecked,stateName,Population (2019 Estimate)
0,2020-03-23,AK,22.0,946.0,NaN,0.0,NaN,968,2020-03-23 20:00:00+00:00,Alaska,731545
1,2020-03-22,AK,22.0,946.0,NaN,0.0,NaN,968,2020-03-22 20:00:00+00:00,Alaska,731545
2,2020-03-21,AK,14.0,758.0,NaN,0.0,NaN,772,2020-03-21 20:00:00+00:00,Alaska,731545
3,2020-03-20,AK,12.0,686.0,NaN,NaN,NaN,698,2020-03-20 20:00:00+00:00,Alaska,731545
4,2020-03-19,AK,6.0,400.0,NaN,NaN,NaN,406,2020-03-19 20:00:00+00:00,Alaska,731545
...,...,...,...,...,...,...,...,...,...,...,...
944,2020-03-11,WY,0.0,NaN,NaN,NaN,NaN,0,2020-03-11 20:00:00+00:00,Wyoming,578759
945,2020-03-10,WY,0.0,0.0,0.0,NaN,NaN,0,2020-03-10 20:00:00+00:00,Wyoming,578759
946,2020-03-09,WY,0.0,NaN,NaN,NaN,NaN,0,2020-03-09 20:00:00+00:00,Wyoming,578759
947,2020-03-08,WY,0.0,NaN,NaN,NaN,NaN,0,2020-03-08 20:00:00+00:00,Wyoming,578759


In [48]:
perspective.PerspectiveWidget(
    state_covid_with_population,
    columns=["positive % total tested", "positive", "total"],
    row_pivots=["date"],
    column_pivots=["stateName"],
    filters=[["positive % total tested", "<", 100]],
    computed_columns=['"positive" % "total" as "positive % total tested"']
)

PerspectiveWidget(column_pivots=['stateName'], columns=['positive % total tested', 'positive', 'total'], compu…

In [44]:
perspective.PerspectiveWidget(
    state_covid_with_population,
    columns=["positive % total tested"],
    plugin="y_line",
    row_pivots=["date"],
    column_pivots=["stateName"],
    filters=[["positive % total tested", "<", 100]],
    computed_columns=['"positive" % "total" as "positive % total tested"']
)

PerspectiveWidget(column_pivots=['stateName'], columns=['positive % total tested'], computed_columns=['"positi…

In [49]:
# Join together county deaths + confirmed dataset
county_covid_confirmed_with_deaths = confirmed_df.join(deaths_df["Deaths"])

In [50]:
county_covid_confirmed_with_deaths

,countyFIPS,County,State,stateFIPS,Date,Confirmed,stateName,Deaths
0,0,Statewide Unallocated,AL,1,1/22/2020,0,Alabama,0
1,1003,Baldwin County,AL,1,1/22/2020,0,Alabama,0
2,1015,Calhoun County,AL,1,1/22/2020,0,Alabama,0
3,1017,Chambers County,AL,1,1/22/2020,0,Alabama,0
4,1043,Cullman County,AL,1,1/22/2020,0,Alabama,0
...,...,...,...,...,...,...,...,...
72036,56021,Laramie County,WY,56,3/22/2020,6,Wyoming,0
72037,56025,Natrona County,WY,56,3/22/2020,1,Wyoming,0
72038,56029,Park County,WY,56,3/22/2020,1,Wyoming,0
72039,56033,Sheridan County,WY,56,3/22/2020,4,Wyoming,0


In [51]:
county = perspective.Table(county_covid_confirmed_with_deaths)

In [53]:
perspective.PerspectiveWidget(
    county,
    aggregates={
        "Death Rate (%)": "last by index",
        "Deaths": "high",
        "Confirmed": "high",
        "stateName": "unique",
        "State": "unique",
        "County": "unique"
    },
    columns=["Death Rate (%)", "Deaths", "Confirmed", "stateName"],
    row_pivots=["County"],
    sort=[["Deaths", "desc"]],
    filters=[["County", "!=", "Statewide Unallocated"]],
    computed_columns=['"Deaths" % "Confirmed" as "Death Rate (%)"']
)

PerspectiveWidget(aggregates={'Death Rate (%)': 'last by index', 'Deaths': 'high', 'Confirmed': 'high', 'state…

In [54]:
county_covid_with_metadata = county_covid_confirmed_with_deaths \
    .merge(county_population_df[["countyFIPS", "Population (2018 Estimate)"]], on="countyFIPS") \
    .merge(county_unemployment_df[["countyFIPS", "Unemployment Rate (2018)", "Unemployed (2018)", "Employed (2018)", "Civilian Labor Force (2018)", "Median Household Income (2018)"]], on="countyFIPS") \

In [55]:
county_covid_with_metadata.dtypes

countyFIPS                          int64
County                             object
State                              object
stateFIPS                           int64
Date                               object
Confirmed                           int64
stateName                          object
Deaths                              int64
Population (2018 Estimate)        float64
Unemployment Rate (2018)          float64
Unemployed (2018)                 float64
Employed (2018)                   float64
Civilian Labor Force (2018)       float64
Median Household Income (2018)    float64
dtype: object

In [56]:
county_with_metadata = perspective.Table(county_covid_with_metadata)

In [57]:
perspective.PerspectiveWidget(
    county_with_metadata,
    aggregates={
        "Death Rate (%)": "last by index",
        "Deaths": "high",
        "Confirmed": "high",
        "stateName": "unique",
        "State": "unique",
        "County": "unique",
        "Unemployment Rate (2018)": "high",
        "Unemployed (2018)": "high",
        "Employed (2018)": "high",
        "Civilian Labor Force (2018)": "high",
        "Population (2018 Estimate)": "high",
        "Median Household Income (2018)": "unique"
    },
    columns=["Death Rate (%)", "Deaths", "Confirmed", "stateName"],
    row_pivots=["County"],
    sort=[["Deaths", "desc"]],
    filters=[["County", "!=", "Statewide Unallocated"]],
    computed_columns=[
        '"Deaths" % "Confirmed" as "Death Rate (%)"',
        '"Confirmed" % "Population (2018 Estimate)" as "Confirmed % Population"'])

PerspectiveWidget(aggregates={'Death Rate (%)': 'last by index', 'Deaths': 'high', 'Confirmed': 'high', 'state…